In [22]:
library('igraph')

In [23]:
fb_combined = read_graph("facebook_combined.txt", format='ncol', directed=FALSE)
fb_graph = as_edgelist(fb_combined)
fb_edgelist = graph_from_edgelist(fb_graph, directed=FALSE)

graph_415 = make_ego_graph(fb_edgelist, nodes=c('414'))
degree_list = degree(graph_415[[1]])
Nr = which(degree_list == 24)

# Question 16
cat(paste("Number of users who we want to recommend friends to:", length(Nr)))

Number of users who we want to recommend friends to: 11

In [24]:
# Question 17
mean_accuracy = function(method) {
    mean_accuracies = c()
    for (node_i in Nr) {
        accuracies = c()
        for (iteration in c(1:10)) {
            temp_g = graph_415[[1]]
            Ri = c()
            neighbors_i = neighbors(temp_g, node_i)
            for (neighbor in neighbors_i) {
                if (runif(1,0,1) <= 0.25) {
                    temp_g = delete_edges(temp_g, edge(neighbor, node_i))
                    Ri = c(Ri, neighbor)
                }
            }
            updated_neighbors_i = setdiff(neighbors_i, Ri)
            updated_neighbors_i = c(updated_neighbors_i, node_i)
            not_neighbors_i = setdiff(V(temp_g), updated_neighbors_i)
            measure = c()
            for (node_j in not_neighbors_i) {
                neighbors_j = neighbors(temp_g, node_j)
                intersected_neighbors = intersect(updated_neighbors_i, neighbors_j)
                if (method == "Common Neighbors") {
                    measure = c(measure, length(intersected_neighbors))
                } else if (method == "Jaccard") {
                    union_neighbors = union(updated_neighbors_i, neighbors_j)
                    measure = c(measure, length(intersected_neighbors)/length(union_neighbors))
                } else if (method == "Adamic-Adar") {
                    sum = 0
                    for (node_k in intersected_neighbors) {
                        sum = sum + 1/log10(length(neighbors(temp_g, node_k)))
                    }
                    measure = c(measure, sum)
                }
            }
            idx = sort(measure, decreasing=TRUE, index.return=TRUE)$ix
            Pi = not_neighbors_i[idx[1:length(Ri)]]
            accuracies = append(accuracies, length(intersect(Ri, Pi))/length(Ri))
        }
        mean_accuracies = append(mean_accuracies, mean(accuracies))
    }
    return (mean(mean_accuracies))
}

In [28]:
cat(paste("Average accuracy of common neighbors mearuse is", mean_accuracy("Common Neighbors")))
cat("\n")
cat(paste("Average accuracy of Jaccard mearuse is", mean_accuracy("Jaccard")))
cat("\n")
cat(paste("Average accuracy of Adamic-Adar mearuse is", mean_accuracy("Adamic-Adar")))

Average accuracy of common neighbors mearuse is 0.859908828545192
Average accuracy of Jaccard mearuse is 0.829422799422799
Average accuracy of Adamic-Adar mearuse is 0.866962481962482